In [570]:
import requests
import pandas as pd
import json
import os
import geopandas as gpd
import numpy as np
import time

In [571]:
cntryc = "zm"

# Define path and name of the file#

ROOT_DIR = os.path.abspath(os.curdir)
admin_path = ROOT_DIR + "\\" + 'Input_Data' 
admin_name = "{}_admin.gpkg".format(cntryc)

# Define path and name of the file
mg_path = ROOT_DIR + "\\" + 'Input_Data' + '\\' + 'VIDA_Oct_18_2023'
mg_name = "VIDA_all_settlements_18102023.gpkg"

# Define path and name of the file
clusters_path = ROOT_DIR + "\\" + 'Input_Data'
clusters_name = "{}-2-0_0_0_0_1_0.csv".format(cntryc)
#clusters_name = "mw-2-0_0_0_0_1_0.csv"

# Path of solar resource data
path = ROOT_DIR + "\\" + 'Input_Data' + '\\' + "{}-2-pv.csv".format(cntryc)

# Path of result files
outpath= ROOT_DIR + "\\" + 'Output_Data'
#outpath= r"C:\Users\alexl\Dropbox\Self-employment\WBG\Work\GEP\WHO_HF\Literature\Haiti docs"

## Adding extraction of only last year (2019) of data

def read_environmental_data(path):
    ghi_curve = pd.read_csv(path, usecols=[3], skiprows=3).values  # * 1000
    ghi_curve = ghi_curve[341879:350643]
    temp = pd.read_csv(path, usecols=[2], skiprows=3).values
    temp = temp[341879:350643]

    return ghi_curve, temp

ghi_curve, temp = read_environmental_data(path)

# Create a new geo-dataframe
#health_gdf = gpd.read_file(mg_path + "\\" + mg_name)
mg_gdf = gpd.read_file(mg_path + "\\" + mg_name)
MG = gpd.GeoDataFrame({
    'Lat': mg_gdf['Lat'][:30],
    'Lon': mg_gdf['Lon'][:30],
    'GHI': mg_gdf['GHI'][:30]
})

def normalised_ghi_location(ghi, ghi_curve,temp):
    pv_capacity=1 #1kW
    k_t=0.005
    ghi = ghi_curve * ghi / ghi_curve.sum()
    t_cell = temp + 0.0256 * ghi  # PV cell temperature
    pv_gen = pv_capacity * 1 * ghi * (1 - k_t * (t_cell - 25)) #adjusted from 0.9 to 1
    
    return pv_gen

In [572]:
print(MG)

          Lat        Lon          GHI
0   -9.890335  31.250981  2032.684937
1  -14.486753  30.866770  1908.004578
2  -13.495112  25.903903  2007.907740
3  -13.334729  32.929618  1991.628585
4  -13.485709  25.893190  2006.690999
5  -10.936067  30.092136  2053.231240
6  -11.096882  30.217005  2063.054430
7   -9.955222  29.963595  1968.787399
8  -12.402135  33.325680  1969.410787
9  -10.438464  32.058317  2030.129883
10 -13.405778  29.873316  1952.875948
11 -12.950658  27.227738  2000.113484
12  -9.128762  31.806709  1980.556641
13  -8.919092  31.628625  1975.459188
14 -15.083357  30.194540  1963.623810
15 -12.786790  26.516036  1992.900024
16 -10.942484  30.142151  2039.056890
17 -14.417850  30.675089  1904.777574
18  -9.314293  31.801365  1991.884225
19 -10.720403  30.384133  2032.196223
20 -11.896564  33.276348  1938.938241
21 -15.464722  23.678678  2072.976898
22  -9.390708  31.674215  1973.025212
23 -12.104030  33.167758  1947.680952
24 -12.275753  30.575803  2025.392236
25 -10.72931

In [573]:
#Define dataframe with the PV generation from the minigrid opt code for future comparision
start = time.time()
PV_gen_1 = MG.apply(lambda row: normalised_ghi_location(row['GHI'],ghi_curve,temp), axis=1)
end = time.time()
yielded_energy_PV_1 = [sum(inner_list[0] for inner_list in row) for row in PV_gen_1]
yielded_energy_PV_1 = pd.DataFrame(yielded_energy_PV_1)
yielded_energy_PV_1.columns = ['Yielded annual Energy dataset 1']
print(PV_gen_1)
print()
print(yielded_energy_PV_1)
print((end - start)/60, "min")

0     [[0.0], [0.0], [0.0], [0.005226770164387453], ...
1     [[0.0], [0.0], [0.0], [0.0049061719342367765],...
2     [[0.0], [0.0], [0.0], [0.005163059046669423], ...
3     [[0.0], [0.0], [0.0], [0.005121199459698265], ...
4     [[0.0], [0.0], [0.0], [0.00515993036517671], [...
5     [[0.0], [0.0], [0.0], [0.005279602128585769], ...
6     [[0.0], [0.0], [0.0], [0.005304861096079645], ...
7     [[0.0], [0.0], [0.0], [0.0050624665230649055],...
8     [[0.0], [0.0], [0.0], [0.005064069476693023], ...
9     [[0.0], [0.0], [0.0], [0.005220200198818243], ...
10    [[0.0], [0.0], [0.0], [0.0050215524340976895],...
11    [[0.0], [0.0], [0.0], [0.005143017199766509], ...
12    [[0.0], [0.0], [0.0], [0.0050927294930732106],...
13    [[0.0], [0.0], [0.0], [0.00507962210104655], [...
14    [[0.0], [0.0], [0.0], [0.005049189069343888], ...
15    [[0.0], [0.0], [0.0], [0.0051244687897874855],...
16    [[0.0], [0.0], [0.0], [0.005243154756881069], ...
17    [[0.0], [0.0], [0.0], [0.00489787414170879

In [574]:
#API function to retrieve PV generation from renewable ninja
token = '963a6e0fc004cb5d98c67c0c17196c594f306c7b'
api_base = 'https://www.renewables.ninja/api/'

s = requests.session()
# Send token header with each request
s.headers = {'Authorization': 'Token ' + token}


def get_pv_data(lat, lon):
    url = api_base + 'data/pv'
    args = {
        'lat': lat,
        'lon': lon,
        'date_from': '2019-01-01',
        'date_to': '2019-12-31',
        'dataset': 'merra2',
        'capacity': 1.0,
        'system_loss': 0,
        'tracking': 0,
        'tilt': 35,
        'azim': 180,
        'format': 'json',
        'raw': 'true'
    }
    r = s.get(url, params=args)
    # Parse JSON to get a pandas.DataFrame of data and dict of metadata
    parsed_response = json.loads(r.text)
    data = pd.read_json(json.dumps(parsed_response['data']), orient='index')
    metadata = parsed_response['metadata']
    
    electricity = pd.DataFrame(data['electricity'].values)
    electricity_data= electricity.transpose()

    return electricity_data

In [575]:
start = time.time()
PV_gen_2=pd.DataFrame()

for index, row in MG.iterrows():
    lat = row['Lat']
    lon = row['Lon']
    
    pv_data = get_pv_data(lat, lon)
    PV_gen_2=pd.concat([PV_gen_2,pv_data])

    #print(PV_gen_2)
    #print()
    #print(yielded_energy_PV_2)

yielded_energy_PV_2=PV_gen_2.sum(axis=1)
yielded_energy_PV_2=yielded_energy_PV_2.to_frame()
yielded_energy_PV_2.columns=['Yielded annual Energy dataset 2']
print(yielded_energy_PV_2)
end = time.time()
print((end - start)/60, "min")


   Yielded annual Energy dataset 2
0                         1878.647
0                         1909.197
0                         1930.197
0                         1919.736
0                         1929.875
0                         1882.032
0                         1887.335
0                         1876.441
0                         1886.225
0                         1869.683
0                         1947.907
0                         1894.688
0                         1847.212
0                         1853.070
0                         1902.409
0                         1901.980
0                         1882.805
0                         1889.604
0                         1845.654
0                         1885.307
0                         1874.381
0                         1963.065
0                         1850.152
0                         1891.087
0                         1942.167
0                         1864.094
0                         1971.831
0                   

In [576]:
yielded_energy_PV_1.reset_index(drop=True, inplace=True)
yielded_energy_PV_2.reset_index(drop=True, inplace=True)
PV_concat=pd.concat([yielded_energy_PV_1,yielded_energy_PV_2], axis=1, ignore_index=True)
PV_concat.columns=['Y1','Y2']
#result = pd.concat([df1, df2], axis=1, ignore_index=True)
print(PV_concat)


             Y1        Y2
0   2010.183357  1878.647
1   1886.892469  1909.197
2   1985.682381  1930.197
3   1969.584682  1919.736
4   1984.479203  1929.875
5   2030.500571  1882.032
6   2040.214222  1887.335
7   1946.998062  1876.441
8   1947.614503  1886.225
9   2007.656790  1869.683
10  1931.263920  1947.907
11  1977.975007  1894.688
12  1958.636140  1847.212
13  1953.595498  1853.070
14  1941.892016  1902.409
15  1970.841950  1901.980
16  2016.484267  1882.805
17  1883.701412  1889.604
18  1969.837473  1845.654
19  2009.700093  1885.307
20  1917.481513  1874.381
21  2050.026036  1963.065
22  1951.188648  1850.152
23  1926.126811  1891.087
24  2002.971962  1942.167
25  2015.565568  1864.094
26  2046.353070  1971.831
27  2074.452311  2022.968
28  1999.429207  1856.439
29  1968.812746  1862.262


In [577]:
describe_output=print(PV_concat.describe())

                Y1           Y2
count    30.000000    30.000000
mean   1979.204730  1897.281667
std      46.839259    41.282313
min    1883.701412  1845.654000
25%    1948.508039  1870.857500
50%    1974.408479  1886.780000
75%    2010.062541  1917.101250
max    2074.452311  2022.968000
